# Concurrency: race conditions; mutual exclusion
_COSC 208, Introduction to Computer Systems, Fall 2025_

## Shared data

* Recall: each thread has its own stack, but the heap is shared among all threads
* Allows thread's return value stored on the heap to be accessed by the main thread
* Allows multiple threads to access/update the same value

_Example_

In [28]:
/* 1*/  #include <pthread.h>
/* 2*/  #include <stdio.h>
/* 3*/  #include <stdlib.h>
/* 4*/  void *funcA(void *arg) {
/* 5*/      int *num = (int *)arg;
/* 6*/      printf("funcA got %d\n", *num);
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  void *funcB(void *arg) {
/*10*/      int *num = (int *)arg;
/*11*/      printf("funcB got %d\n", *num);
/*12*/      return NULL;
/*13*/  }
/*14*/  int main() {
/*15*/      int *lucky = malloc(sizeof(int));
/*16*/      *lucky = 13;
/*17*/      pthread_t threadA, threadB;
/*18*/      pthread_create(&threadA, NULL, &funcA, lucky);
/*19*/      pthread_create(&threadB, NULL, &funcB, lucky);
/*20*/      pthread_join(threadA, NULL);
/*21*/      pthread_join(threadB, NULL);
/*22*/  }

funcB got 13
funcA got 13


<p style="height:28em;"></p>

_Another Example_

In [58]:
/* 1*/  #include <pthread.h>
/* 2*/  #include <stdio.h>
/* 3*/  #include <stdlib.h>
/* 4*/  void *funcA(void *arg) {
/* 5*/      int *num = (int *)arg;
/* 6*/      *num *= 3;
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  void *funcB(void *arg) {
/*10*/      int *num = (int *)arg;
/*11*/      *num += 2;
/*12*/      return NULL;
/*13*/  }
/*14*/  int main() {
/*15*/      int *data = malloc(sizeof(int));
/*16*/      *data = 5;
/*17*/      pthread_t threadA, threadB;
/*18*/      pthread_create(&threadA, NULL, &funcA, data);
/*19*/      pthread_create(&threadB, NULL, &funcB, data);
/*20*/      pthread_join(threadA, NULL);
/*21*/      pthread_join(threadB, NULL);
/*22*/      printf("data is now %d\n", *data);
/*23*/  }

data is now 17


* Non-determinisim: program's output depends on the order in which threads are executed
* _How can we eliminate this non-determinism?_
    * Create and join threadA before creating and joining threadB – eliminates parallelism
    * Don't perform operations where order matters

## Race conditions

_Example_

In [64]:
/* 1*/  #include <pthread.h>
/* 2*/  #include <stdio.h>
/* 3*/  #include <stdlib.h>
/* 4*/  void *funcA(void *arg) {
/* 5*/      int *num = (int *)arg;
/* 6*/      *num += 3;
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  void *funcB(void *arg) {
/*10*/      int *num = (int *)arg;
/*11*/      *num += 2;
/*12*/      return NULL;
/*13*/  }
/*14*/  int main() {
/*15*/      int *data = malloc(sizeof(int));
/*16*/      *data = 5;
/*17*/      pthread_t threadA, threadB;
/*18*/      pthread_create(&threadA, NULL, &funcA, data);
/*19*/      pthread_create(&threadB, NULL, &funcB, data);
/*20*/      pthread_join(threadA, NULL);
/*21*/      pthread_join(threadB, NULL);
/*22*/      printf("data is now %d\n", *data);
/*23*/  }

data is now 10


<p style="height:1em;"></p>

_Another example_

In [72]:
/* 1*/  #include <pthread.h>
/* 2*/  #include <stdio.h>
/* 3*/  #include <stdlib.h>
/* 4*/  void *funcA(void *arg) {
/* 5*/      int *num = (int *)arg;
/* 6*/      for (int i = 0; i < 100000; i++) {
/* 7*/          *num += 1;
/* 8*/      }
/* 9*/      return NULL;
/*10*/  }
/*11*/  void *funcB(void *arg) {
/*12*/      int *num = (int *)arg;
/*13*/      for (int i = 0; i < 100000; i++) {
/*14*/          *num += 1;
/*15*/      }
/*16*/      return NULL;
/*17*/  }
/*18*/  int main() {
/*19*/      int *data = malloc(sizeof(int));
/*20*/      *data = 0;
/*21*/      pthread_t threadA, threadB;
/*22*/      pthread_create(&threadA, NULL, &funcA, data);
/*23*/      pthread_create(&threadB, NULL, &funcB, data);
/*24*/      pthread_join(threadA, NULL);
/*25*/      pthread_join(threadB, NULL);
/*26*/      printf("data is now %d\n", *data);
/*27*/  }

data is now 110378


* Data race: two threads attempt to write to the same memory location
* Race condition: simultaneous execution of two operations gives an incorrect result
* Add operation is compiled into multiple assembly instructions – `ldr`, `ldr`, `add`, `str`
* Simultaneous execution can lead to incorrect result
  
| Core 0 (Thread A) | Core 1 (Thread B) |
|-------------------|-------------------|
| `ldr x0, [sp,#8]` | `ldr x0, [sp,#8]` |
| `ldr w1, [x0]`    | `ldr w1, [x0]`    |
| `add w1, w1, #1`  | `add w1, w1, #1`  |
| `str w1, [x0]`    | `str w1, [x0]`    |

* Issue occurs as long as both threads load the value before either thread stores

| Core 0 (Thread A) | Core 1 (Thread B) |
|-------------------|-------------------|
| `ldr x0, [sp,#8]` |                   |
| `ldr w1, [x0]`    | `ldr x0, [sp,#8]` |
| `add w1, w1, #1`  | `ldr w1, [x0]`    |
| `str w1, [x0]`    | `add w1, w1, #1`  |
|                   | `str w1, [x0]`    |

* Critical section: section of code that causes a race condition

<p style="height:30em;"></p>

Q1: _What are all possible outputs this program may produce?_

In [ ]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
struct transaction {
    int *balance;
    int amount;
};
void *deposit(void *arg) {
    struct transaction *trans = (struct transaction *)arg;
    int *balance = trans->balance;
    *balance += trans->amount;
    return NULL;
}
void *withdraw(void *arg) {
    struct transaction *trans = (struct transaction *)arg;
    int *balance = trans->balance;
    *balance -= trans->amount;
    return NULL;
}
int main() {
    pthread_t thrA, thrB;
    int *balance = malloc(sizeof(int));
    *balance = 250;
    struct transaction transA = { balance, 50 };
    struct transaction transB = { balance, 100 };
    pthread_create(&thrA, NULL, &deposit, &transA);
    pthread_create(&thrB, NULL, &withdraw, &transB);
    pthread_join(thrB, NULL);
    pthread_join(thrA, NULL);
    printf("Balance: $%d\n", *balance);
}

```
Balance: $200
```
OR
```
Balance: $300
```
OR
```
Balance: $150
```

Q2: _What is a possible interleaving of threads that can produce each output?_

| Core 0 (Thread A)     | Core 1 (Thread B)     |
|-----------------------|-----------------------|
| `ldr balance --> reg` |                       |
| `add reg + 50`        |                       |
| `str reg --> balance` |                       |
|                       | `ldr balance --> reg` |
|                       | `sub reg - 100`       |
|                       | `str reg --> balance` |

| Core 0 (Thread A)     | Core 1 (Thread B)     |
|-----------------------|-----------------------|
| `ldr balance --> reg` |                       |
| `add reg + 50`        |                       |
|                       | `ldr balance --> reg` |
|                       | `sub reg - 100`       |
|                       | `str reg --> balance` |
| `str reg --> balance` |                       |


| Core 0 (Thread A)     | Core 1 (Thread B)     |
|-----------------------|-----------------------|
|                       | `ldr balance --> reg` |
|                       | `sub reg - 100`       |
| `ldr balance --> reg` |                       |
| `add reg + 50`        |                       |
| `str reg --> balance` |                       |
|                       | `str reg --> balance` |


<p style="height:20em;"></p>

Q3: _Where are the critical sections?_

```
*balance += trans->amount;
```
AND
```
*balance -= trans->amount;
```

<p style="height:2em;"></p>

## Mutual exclusion

* Allow only one thread at a time to execute a critcal section
    * If another thread is in the middle of the critical section, then a thread cannot start executing the critical section until the other thread has finished executing the critical section
* Mutual exclusion lock (mutex) is a variable with two states: unlocked (initial state) or locked
    ```
    pthread_mutex_t mutex;
    pthread_mutex_init(&mutex);
    ```
* Lock mutex at the beginning of a critical section and unlock mutex at the end of a critical section, e.g.:
    ```
    pthread_mutex_lock(&mutex);
    *num += 1;
    pthread_mutex_unlock(&mutex);
    ```
* If a thread calls `lock` and the mutex is already locked, then the thread must wait until the mutex is unlocked (by another thread)
* Both threads must use the same lock – declare as global variable or declare in main thread and pass to both threads

<p style="height:20em;"></p>

Q4: _A program contains the following code:_

In [ ]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
pthread_mutex_t totalmutex;
void *dec(void *arg) {
    int *total = (int *)arg;
    pthread_mutex_lock(&totalmutex);
    *total -= 1;
    pthread_mutex_unlock(&totalmutex);
    return NULL;
}
void *inc(void *arg) {
    int *total = (int *)arg;
    *total += 1;
    return NULL;
}
void *zero(void *arg) {
    int *total = (int *)arg;
    *total = 0;
    return NULL;
}
int main() {
    pthread_mutex_init(&totalmutex, 0);
    int *result = malloc(sizeof(int));
    *result = 2;
    pthread_t thrA, thrB;
    // TODO: create and join threads
    printf("%d\n", *result);
}

_For each of the following code snippets: if the `TODO` in `main` was replaced with the provided code snippet, list all possible outputs the program could produce._

In [ ]:
pthread_create(&thrA, NULL, &inc, result);
pthread_create(&thrB, NULL, &inc, result);
pthread_join(&thrA);
pthread_join(&thrB);

`4`  or  `3`

<p style="height:10em;"></p>

In [ ]:
pthread_create(&thrA, NULL, &dec, result);
pthread_create(&thrB, NULL, &dec, result);
pthread_join(&thrA);
pthread_join(&thrB);

`0`

<p style="height:10em;"></p>

In [ ]:
pthread_create(&thrA, NULL, &inc, result);
pthread_create(&thrB, NULL, &dec, result);
pthread_join(&thrB);
pthread_join(&thrA);

`2`  or  `3`  or  `1`

<p style="height:10em;"></p>

In [ ]:
pthread_create(&thrA, NULL, &inc, result);
pthread_create(&thrB, NULL, &zero, result);
pthread_join(&thrB);
pthread_join(&thrA);

`0`  or  `1`  or  `3`